# Star-Wars Aura Demo Notebook  v 1.2
## Cypher Examples Using Neo4j Python Driver


## Overview
To demonstrate how to read and write data to a Neo4j instance running in AuraDB using the Neo4j Python Driver.

### Created: September 11, 2022
#### Latest version 1.2 - February 3, 2023

### Author:  Mark Quinsland 
mark.quinsland@neo4j.com

### Additional resources:
Some of this material is based on the excellent, highly-recommended "Bite-Sized Neo4j for Data Scientiests" series of notebooks and youtube clips by former Neo4j data scientist CJ Sullivan. For some of the notebooks in CJ's series, the official Neo4j Python driver named 'neo4j' is used.  In others, the community-supported 'py2neo' driver is used.  
Sample notebooks and tutorials may be found here:  

- https://github.com/cj2001/bite_sized_data_science
- https://youtu.be/Niys6g6NFfw?list=PL9Hl4pk2FsvVShoT5EysHcrs-hyCsXaWC
- https://medium.com/@cj2001


## A Note about Neo4j's family of Python Drivers.

There are 3 commonly used Python drivers for Neo4j.  This notebook uses the official Neo4j Python driver. 

### neo4j  driver (Official Neo4j Python Driver)
This is the official, supported Neo4j Python driver and is continually being updated to utilize Neo4j's latest features.  The driver is designed to work with a wide variety of Neo4jDB instances including on-prem, AWS, GCP, Azure, and AuraDB.  It greatly simplifies connectivity to Neo4j clusters.
- https://neo4j.com/docs/api/python-driver/current/
- https://neo4j.com/docs/python-manual/current/


### graphdatascience driver ( GDS Clients only)
Neo4j recently released a new Python client named 'graphdatascience' that supports additional Graph Data Science features.  This client should only be used to access Neo4j servers where the GDS library has been installed.    Documentation and examples using the new graphdatascience library are available from the following links.
- https://pypi.org/project/graphdatascience/
- https://neo4j.com/developer-blog/get-started-with-neo4j-gds-python-client/

### py2neo (community supported driver)

Py2neo is a simple-to-use library and was a very popular alternative to the official Neo4j driver. However, it does not provide support for enterprise Neo4j features or for recent Neo4j versions.  
https://py2neo.org/v4/




### Install Neo4j Python Driver - pip install neo4j

In [12]:
# use pip install neo4j

from neo4j import GraphDatabase
from neo4j.exceptions import ServiceUnavailable
import pandas as pd



## Connection Values 
- downloaded at time of instance creation!
#### Note: Take appropriate measures to keep authorization information secret.  Values are hardcoded here for simplicity.

In [30]:
#aura connection details - downloaded at time of instance creation!

NEO4J_URI="neo4j+s://d44b75fb.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="UmFkV80lKjANvfimU3RBVpdeXSI9asY1Q31gOhWUZV4"
AURA_INSTANCENAME="Instance01"


In [31]:
# use helper class to handle Neo4j connection tasks
conn = Neo4jConnection(uri=NEO4J_URI, user=NEO4J_USERNAME, pwd=NEO4J_PASSWORD)

In [32]:
## simple example to test connection & return results
result = conn.query('MATCH (n) RETURN COUNT(n) AS ct')
# Consume the result object
print(result[0]['ct'])

249


# Cypher Query Examples

## Execute Cypher read query and process results

### Which characters does Han Solo interact with the most?

In [33]:
# load any parameters using a dictionary  

params = {'name': 'Han Solo' }

# cypher - who does Han Solo interact with the most?
query = """MATCH (p:Character {name: $name})-[i:INTERACTS_WITH]-(p2:Character)
           RETURN  p2.name AS otherName, i.count as interactionCount
           ORDER BY interactionCount DESC LIMIT 10
    
"""

result = conn.query(query, parameters=params)

for row in result:
    print (row['otherName'], row['interactionCount'])

Chewbacca 77
Leia Organa 69
C-3PO 54
Luke Skywalker 43
Finn 23
R2-D2 18
Rey 17
LANDO 12
Obi-Wan Kenobi 10
BB8 9


#### Execute Cypher read query and export to DataFrame

In [34]:
# load any parameters 

params = {'name': 'Han Solo'  }

query = """MATCH (p:Character {name: $name})-[i:INTERACTS_WITH]-(p2:Character)
           RETURN  p2.name AS otherName, i.count as interactionCount
           ORDER BY interactionCount DESC LIMIT 10         
"""

# run query to get results
result = conn.query(query, parameters=params)

# create a dataframe from Cypher Resultset  
dtf_data = pd.DataFrame([dict(_) for _ in result])
dtf_data.head(3)


,otherName,interactionCount
0,Chewbacca,77
1,Leia Organa,69
2,C-3PO,54


## Write Data to Neo4j Using Dictionaries

This example uses dictionaries to contain the information about Characters  and their home planets.   It will create (if necessary) a Character node for each dictionary in the array.  It will also create relationships between the Character nodes and the Planet nodes.


In [39]:
params = { }

# create a batch of dictionaries - each item will be processed individually within the batch
bunch_of_rows = [
 {'name':'Mark', 'gender': 'male', 'planet':'Endor','jedi':'true'},
 {'name': 'Abby', 'gender': 'female', 'planet':'Endor', 'jedi':'false'},
 {'name': 'Grokster', 'gender': 'male', 'planet':'Alderaan', 'jedi':'false'}

]
params ['rows'] = bunch_of_rows

query = """

// use UNWIND to process each item in the array
UNWIND $rows AS row
     MERGE (u:Character {name: row.name})
     SET u.gender = row.gender,
         u.jedi = toBoolean(row.jedi)
         
    // now create a relationship from the Character to their home planet
    
     WITH u, row.planet as homePlanet
         // find the planet and create a relationship to it
        MATCH (p:Planet)
            WHERE p.name = homePlanet
        MERGE (u)-[:FROM]->(p)
return count(*) as total
"""

# run query to get results
result = conn.query(query, parameters=params)
print (result)


[<Record total=3>]


### Utility class for Neo4j connection

In [6]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response